In [2]:
import time
from random import random,randint

class Tree:

    label = "Tree"

    def __init__(self):
        self.root = None

    def empty(self):
        return True if self.root is None else False

    def getRoot(self):
        return None if self.root is None else self.makePosition(self.root)

    def setRoot(self,thing):
        if thing is None:
            self.root = None
            return None
        elif isinstance(thing,Tree.Position):
            self.root = thing.unwrap()
        elif not isinstance(thing,Tree._Node):
            self.root = self.makeNode(thing)
        else:
            self.root = thing

        return self.makePosition(self.root)

    def print(self,indent=''):
        if self.root is None:
            print(f'Empty {self.label}')
        else:
            print(f'Instance of {self.label} with structure:')
            self.root.print(indent="  ")

    def search(self,value):
        if self.root is None:
            return None

        found = self.root.search(value)

        return None if found is None else self.makePosition(found)

    def DFT(self,order='preorder',fcn=print):
        if self.root is None:
            return None
        else:
            return self.root.DFT(order,fcn)

    def height():
        return 0 if self.root is None else self.root.height()

    def size():
        return 0 if self.root is None else self.root.size()

    def makeNode(self,value):
        return Tree._Node(tree=self,value=value)

    def makePosition(self,node):
        return Tree.Position(tree=self,node=node)

    class _Node:
        def __init__(self,tree=None,value=None,parent=None):
            self.tree = tree
            self.value = value
            self.parent = None
            # need to put links to children here in derived classes!

        def children(self):
            print('[calling dummy method children of _Node]')
            pass

        def DFT(self,order='preorder',fcn=print):
            if order == 'preorder':
                fcn(self)

            kids = self.children()

            if order == 'inorder':
                if kids:
                    kids[0].DFT(order,fcn)
                fcn(self)
                if kids:
                    kids = kids[1:]

            for kid in kids:
                kid.DFT(order,fcn)

            if order == 'postorder':
                fcn(self)

        def height(self):
            kids = self.children()
            
            maxHt = 0
            for kid in kids:
                maxHt = max(kid.height(),maxHt)

            return maxHt + 1

        def size(self):
            def make_counter():
                count = 0
                def counter(node):
                    nonlocal count
                    count += 1
                    return count
                return counter

            counter = make_counter()
            self.DFT(fcn=counter)
            return counter(None) - 1

        def print(self,indent=''):
            print(f'{indent}{self.value}')
            for kid in self.children():
                kid.print(indent + '   ')

        def search(self,thing):
            if self.value == thing:
                return self
            else:
                for kid in self.children():
                    found = kid.search(thing)
                    if found:
                        return found
                return None

        def addChild(self,child):
            print('[calling dummy method addChild of _Node]')
            pass

        def deleteChild(self,child):
            print('[calling dummy method deleteChild of _Node]')
            pass

        def findChild(self,child):
            if isinstance(child,Tree._Node):
                kids = self.children()
                found = kids.index(child)
                if found:
                    return kids[found]
            else:
                for kid in self.children():
                    if kid.value == thing:
                        return kid
            return None

    class Position:
        def __init__(self,tree=None,node=None):
            self.tree = tree
            self.node = node

        def unwrap(self):
            return self.node

        def wrap(self,node):
            return None if node is None else self.tree.makePosition(node)

        def parent(self):
            return self.wrap(self.node.parent)

        def value(self):
            return self.unwrap().value

        def children(self):
            return [self.wrap(i) for i in self.node.children()]

        def addChild(self,thing):
            thing = thing.unwrap() if isinstance(thing,Tree.Position) else thing
            return self.wrap(self.node.addChild(thing))

        def deleteChild(self,thing):
            thing = thing.unwrap() if isinstance(thing,Tree.Position) else thing
            deleted= self.node.deleteChild(thing)

            return None if not deleted else self.wrap(deleted)

        def search(self,thing):
            thing = self.value() if isinstance(thing,Tree.Position) else thing
            found = self.unwrap().search(thing)

            return None if not found else self.wrap(found)

        def DFT(self,order='preorder',fcn=print):
            return self.unwrap().DFT(order,fcn)

        def print(self,indent=''):
            return self.unwrap().print(indent)

        def sameAs(self,pos):
            return self.unwrap() == pos.unwrap()

class BinaryTree(Tree):

    label = 'BinaryTree'

    def makeNode(self,value):
        return BinaryTree._BinaryTreeNode(tree=self,value=value)

    def makePosition(self,node):
        return BinaryTree.BTPosition(tree=self,node=node)

    class _BinaryTreeNode(Tree._Node):

        def __init__(self,tree=None,value=None,parent=None):
            # Call Tree._Node's __init__ method first:
            super().__init__(tree,value,parent)
            # Now add what we need here:
            self.left = None
            self.right = None

        def children(self):
            kids = []
            if self.left is not None:
                kids.append(self.left)
            if self.right is not None:
                kids.append(self.right)

            return kids

        class InvalidMethod(Exception):
            pass

        # a generic addChild method doesn't make sense for us, so just
        # override and generate an error:
        def addChild(self,child):
            print('Invalid method of BinaryTree addChild.')
            raise self.InvalidMethod


        # ditto for deleteChild:
        def deleteChild(self,child):
            print('Invalid method of BinaryTree deleteChild.')
            raise self.InvalidMethod

        def setLeft(self,thing):
            if isinstance(thing,Tree._Node):
                thing.parent = self
                self.left = thing
                return thing
            else:
                thing = self.tree.makeNode(thing)
                self.setLeft(thing)


        def setRight(self,thing):
            if isinstance(thing,Tree._Node):
                thing.parent = self
                self.right = thing
                return thing
            else:
                thing = self.tree.makeNode(thing)
                self.setRight(thing)

        def add(self,value):
            if self.value is value:
                return False
            elif self.value is None:
                self.value = value
                return BinaryTree.BTPosition(self.left)
            elif value[1] < self.value[1]:
                if self.left:
                    return self.left.add(value)
                else:
                    self.setLeft(value)
                    return BinaryTree.BTPosition(self.left)
            else:
                if self.right:
                    return self.right.add(value)
                else:
                    self.setRight(value)
                    return BinaryTree.BTPosition(self.right)
                
        def find(self,value):
            if self.value[1] == value:
                return BinaryTree._BinaryTreeNode(self.value)
            elif value < self.value[1] and self.left:
                return self.left.find(value)
            elif value > self.value[1] and self.right:
                return self.right.find(value)
            return None
        
        # Find lowest value in a node's subtree
        def minNode(self,node):
            current = node
            # While node has a left child
            while(current.left is not None):
                
                current = current.left
                
            return current
        
        def delete(self,value):
            if value is None:
                return self
            # Key lies in left subtree
            elif (value[1] < self.value[1]):
                if self.left is not None:
                    self.left.delete(value)
            # Key lies in right subtree
            elif value[1] > self.value[1]:
                if self.right is not None:
                    self.right.delete(value)
            # If value is the same as self.value
            elif value[1] == self.value[1]:
                # If node has a right child but no left child
                if self.left is None and self.right:
                    self.value = self.right.value
                    self.right = None
                    return self.value
                # If node has a left child but no right child
                elif self.left and self.right is None:
                    self.value = self.left.value
                    self.left = None
                    return self.value
                # If node has two children (only works when balanced)
                elif self.left and self.right:
                    # Lowest value in a node's subtree (tuple)
                    lowestNode = self.minNode(self)
                    # Overwrites current node's tuple
                    self.value = lowestNode.value
                    print(lowestNode.value)
                    self.left.delete(self.left.value)
                else:
                    self.value = None
                    return self.value
                
        def inorder(self):
            out = []
            if self.left:
                self.left.inorder()
            out.append(self.value)
            if self.right:
                self.right.inorder()
            return out

    class BTPosition(Tree.Position):
        # Nothing special needed for instance variables; just use the parent's.

        def left(self):
            l = self.unwrap().left
            return None if l is None else self.tree.makePosition(l)

        def right(self):
            r = self.unwrap().right
            return None if r is None else self.tree.makePosition(r)

        def setLeft(self,thing):
            return self.tree.makePosition(self.unwrap().setLeft(thing))

        def setRight(self,thing):
            return self.tree.makePosition(self.unwrap().setRight(thing))
        
class BST(BinaryTree):
         
    def add(self,value):
        if value[1] < self.root.value[1]:
            if self.root.left:
                return self.root.left.add(value)
            else:
                self.root.setLeft(value)
        else:
            if self.root.right:
                return self.root.right.add(value)
            else:
                self.root.setRight(value)
    
    def delete(self,value):
        if self.root is None:
            return self.root
        # Key lies in left subtree
        elif value[1] < self.root.value[1]:
            self.root.left.delete(value)
        # Key lies in right subtree
        elif value[1] > self.root.value[1]:
            self.root.right.delete(value)
        # If key is the same as root
        else:
            if self.root.left is None:
                temp = self.root.right
                self.root = None
                return temp
            elif self.root.right is None:
                temp = self.root.left
                root = None
                return temp  
    
    # Finds a node based on it's key value
    def find(self,value):
        if self.root.value[1] == value:
            return BinaryTree._BinaryTreeNode(self.root.value)
        elif value < self.root.value[1] and self.root.left:
            return self.root.left.find(value)
        elif value > self.root.value[1] and self.root.right:
            return self.root.right.find(value)
    
    def next(self,value):
        found = self.find(value)
        # If the node with value is the last node
        if found.right is None:
            return None
        else:
            return BinaryTree._BinaryTreeNode(found.right.value)
    
    def prev(self,value):
        found = self.find(value)
        # If the node with value is the root
        if found.left is None:
            return None
        else:
            return BinaryTree._BinaryTreeNode(found.left)
    
    def inorder(self):
        out = []
        if self.root:
            self.root.left.inorder()
        out.append(self.root.value)
        self.root.right.inorder()
        return out
    
    def sort(self,L):
        []

In [4]:
T = BST()
student = ("Roy Turner",3.5)
T.setRoot(student)
student = ("Big Guy",3.45)
T.root.setLeft(student)
student = ("Orson Welles",3.0)
T.add(student)
student = ("Ben Farman",3.6)
T.add(student)
student = ("Mike Nurmoe",3.49)
T.add(student)
student = ("Jake Nat",3.75)
T.add(student)
student = ("Avery Gosselin",3.55)
T.add(student)
student = ("Jeremy Butler",3.65)
T.add(student)
student = ("Paul Fargo",3.8)
T.add(student)
student = ("Brandon Doonan",3.76)
T.add(student)
student = ("John Doe",3.84)
T.add(student)
student = ("Ian Bowers",3.63)
T.add(student)
student = ("Jack Barclay",3.61)
T.add(student)
student = ("Vincenzo Grutti",3.64)
T.add(student)

T.print()
print("========AFTER DELETION========")
toDelete = ("Ian Bowers",3.63)
T.delete(toDelete)
T.print()

Instance of BinaryTree with structure:
  ('Roy Turner', 3.5)
     ('Big Guy', 3.45)
        ('Orson Welles', 3.0)
        ('Mike Nurmoe', 3.49)
     ('Ben Farman', 3.6)
        ('Avery Gosselin', 3.55)
        ('Jake Nat', 3.75)
           ('Jeremy Butler', 3.65)
              ('Ian Bowers', 3.63)
                 ('Jack Barclay', 3.61)
                 ('Vincenzo Grutti', 3.64)
           ('Paul Fargo', 3.8)
              ('Brandon Doonan', 3.76)
              ('John Doe', 3.84)
========AFTER DELETION========
('Jack Barclay', 3.61)
Instance of BinaryTree with structure:
  ('Roy Turner', 3.5)
     ('Big Guy', 3.45)
        ('Orson Welles', 3.0)
        ('Mike Nurmoe', 3.49)
     ('Ben Farman', 3.6)
        ('Avery Gosselin', 3.55)
        ('Jake Nat', 3.75)
           ('Jeremy Butler', 3.65)
              ('Jack Barclay', 3.61)
                 None
                 ('Vincenzo Grutti', 3.64)
           ('Paul Fargo', 3.8)
              ('Brandon Doonan', 3.76)
              ('John Doe'

For reference of how print() ouputs the tree:

In [3]:
T = BST()
T.setRoot('root')
T.root.setLeft('left')
T.root.setRight('right')
T.root.left.setLeft('leftleft')
T.root.left.setRight('leftright')
T.root.right.setLeft('rightleft')
T.root.right.setRight('rightright')
T.print()

Instance of BinaryTree with structure:
  root
     left
        leftleft
        leftright
     right
        rightleft
        rightright
